In [ ]:
import pypdf
import re
from typing import List
import os
from FlagEmbedding import BGEM3FlagModel

import chromadb
import tempfile
import numpy as np

os.environ['CURL_CA_BUNDLE'] = ''

In [2]:
reader = pypdf.PdfReader("ai_doc.pdf")

In [3]:
all_text = ""
for i in range (0, 8):
    text = reader.pages[i].extract_text()
    all_text += text

In [4]:
def clean_text(text: str) -> str:
    """Clean and normalize text content"""
    # Removing multiple lines
    text = re.sub(r"\n+", "\n", text)

    # Removing multiple spaces
    text = re.sub(r"\s+", " ", text)

    # Remove HTML comments 
    text = re.sub(r"<!--.?-->", "", text)
    text = re.sub(r"&", "u", text)

    return text

In [5]:
def split_into_chunks(text: str, max_chunk_size: int = 1024) -> List[str]:
    """Creates chunks with size of max_chunk_size parameter from the document."""
    chunks = []
    sentences = re.split(r'(?<=[.!?])\s+', text) # Splitting sentences from the document
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence_size = len(sentence)
        if current_size+sentence_size > max_chunk_size and current_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_size = 0
        
        current_chunk.append(sentence)
        current_size += sentence_size
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

In [6]:
clean = clean_text(text=all_text)

In [7]:
chunks = split_into_chunks(text=clean, max_chunk_size=600)

In [8]:
for i, chunk in enumerate(chunks):
    filename = f"chunk_{i}.txt"
    filepath = f"/home/murad/Documents/contextual_embeddings/chunks/{filename}"

    try:
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(chunk)
    except Exception as e:
        print(f"Error saving chunk to {filepath}: {str(e)}.")

In [10]:
# Then load your model
model = BGEM3FlagModel("BAAI/bge-m3", 
                      use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
embeddings_1 = model.encode(chunks, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
query = ["Turinq testi nədir?"]
query_embedding = model.encode(sentences=query,
                               batch_size=12,
                               max_length=1024)["dense_vecs"]
query_embedding = query_embedding.tolist()

In [ ]:
# Create a temporary directory with write permissions
temp_dir = tempfile.mkdtemp()
print(f"Using ChromaDB directory: {temp_dir}")

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=temp_dir)

# Create a collection WITHOUT an embedding function
collection = client.create_collection(
    name="pdf_chunks",
    metadata={"description": "PDF chunks with BGE-M3 embeddings"}
)

# Prepare your data
document_ids = [f"chunk_{i}" for i in range(len(chunks))]
metadatas = [{"source": f"chunk_{i}.txt"} for i in range(len(chunks))]

# Convert numpy arrays to lists if needed
embeddings_list = []
for emb in embeddings_1:
    if isinstance(emb, np.ndarray):
        embeddings_list.append(emb.tolist())
    else:
        embeddings_list.append(emb)

# Add documents with pre-computed embeddings
collection.add(
    documents=chunks,            # The document texts
    ids=document_ids,            # Unique IDs
    metadatas=metadatas,         # Metadata
    embeddings=embeddings_list   # Your pre-computed embeddings
)

print(f"Successfully stored {len(docs)} documents with embeddings in ChromaDB")
print(f"Collection name: pdf_chunks")
print(f"Database location: {temp_dir}")

Using ChromaDB directory: /tmp/tmpirl3m0rc
Successfully stored 37 documents with embeddings in ChromaDB
Collection name: pdf_chunks
Database location: /tmp/tmpirl3m0rc


In [20]:
collection.query(query_embeddings=query_embedding,
                 n_results=10,
                 include=["documents", "distances", "metadatas"])

{'ids': [['chunk_11',
   'chunk_31',
   'chunk_33',
   'chunk_12',
   'chunk_25',
   'chunk_1',
   'chunk_3',
   'chunk_6',
   'chunk_36',
   'chunk_24']],
 'embeddings': None,
 'documents': [['Turinq, bir kompüterin süni zəkaya sahib ola biləcəyini və hakimi çaşdıracaq qədər inandırıcı ola biləcəyini iddia edir. Hakim, insanla yoxsa kompüterlə danışdığını anlamayacaq. Bu testə Turinq testi deyilir.Turinq testinin məqsədi və əhəmiyyəti Turinq testi, bir maşının insan səviyyəsində zəkaya sahib olub olmadığını ölçmək üçün bir meyardır. Testin məqsədi, bir maşının davranışının insan davranışına o qədər bənzəyib-bənzəmədiyini müəyyən etməkdir ki, onu ayırd etmək mümkün olmasın.',
   'Turinq testi, süni zəka tədqiqatları sahəsində mühüm bir rol oynamış və bu sahədə bir çox müzakirəyə səbəb olmuşdur. Turinq testinin tənqidləri Turinq testi, süni zəkanın ölçülməsi üçün bir meyar olaraq tənqid edilmişdir. Bəzi tənqidçilər, testin yalnız bir maşının insan kimi davranma qabiliyyətini ölçdüyünü v